# Semantic Segmentation Project

In this project, we'll label the pixels of a road in images using a Fully Convolutional Network (FCN).

## Dataset
We need to downloaded the Kitti Road dataset from [here](http://www.cvlibs.net/download.php?file=data_road.zip) . After extracting the dataset into the data folder we'll have the folder [data_road](data/data_road) with all the training and test images.


 - [dave-msk/CarND-Semantic-Segmentation-Submission](https://github.com/dave-msk/CarND-Semantic-Segmentation-Submission)
 

In [1]:
import os.path
import shutil
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion(
    '1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

TensorFlow Version: 1.3.0


In [ ]:
# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

In [2]:
def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'
    vgg_proto = tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    vgg = sess.graph
    image_input = vgg.get_tensor_by_name(vgg_input_tensor_name)
    keep_prob = vgg.get_tensor_by_name(vgg_keep_prob_tensor_name)
    layer3_out = vgg.get_tensor_by_name(vgg_layer3_out_tensor_name)
    layer4_out = vgg.get_tensor_by_name(vgg_layer4_out_tensor_name)
    layer7_out = vgg.get_tensor_by_name(vgg_layer7_out_tensor_name)
    return image_input, keep_prob, layer3_out, layer4_out, layer7_out

tests.test_load_vgg(load_vgg, tf)


test_load_vgg
Tests Passed


In [3]:
def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer7_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer3_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    decode_layer1_preskip0 = tf.layers.conv2d_transpose(vgg_layer7_out, 512, (2, 2), (2, 2), name='decode_layer1_preskip0')
    decode_layer1_preskip1 = tf.layers.conv2d(vgg_layer4_out, 512, (1, 1), (1, 1), name='decode_layer1_preskip1')
    decode_layer1_out = tf.add(decode_layer1_preskip0, decode_layer1_preskip1, name='decode_layer1_out')
    decode_layer2_preskip0 = tf.layers.conv2d_transpose(decode_layer1_out, 256, (2, 2), (2, 2), name='decode_layer2_preskip0')
    decode_layer2_preskip1 = tf.layers.conv2d(vgg_layer3_out, 256, (1, 1), (1, 1), name='decode_layer2_preskip1')
    decode_layer2_out = tf.add(decode_layer2_preskip0, decode_layer2_preskip1, name='decode_layer2_out')
    decode_layer3_out = tf.layers.conv2d_transpose(decode_layer2_out, 128, (2, 2), (2, 2), name='decode_layer3_out')
    decode_layer4_out = tf.layers.conv2d_transpose(decode_layer3_out, 64, (2, 2), (2, 2), name='decode_layer4_out')
    decode_layer5_out = tf.layers.conv2d_transpose(decode_layer4_out, num_classes, (2, 2), (2, 2), name='fcn_out')
    return decode_layer5_out

tests.test_layers(layers)


Tests Passed
